## Practical Bioinformatics. Homework 1

### Biopython: Installation


In [1]:
!pip install biopython
import Bio

     |████████████████████████████████| 2.3 MB 10.9 MB/s 


### GenBank and EMBOSS

Let's find the bacterial genome (in my case, its accession number is CP045149.1) in the GenBank and download it as FASTA format.

An accession number in bioinformatics is a unique identifier given to a DNA or protein sequence record to allow for tracking of different versions of that sequence record and the associated sequence over time in a single data repository.

In [2]:
from Bio import Entrez
Entrez.email = "Your.Name.Here@example.org"
handle = Entrez.efetch(db="nucleotide", id="CP045149.1", rettype="fasta", retmode="text")
print(handle.readline().strip())
handle.close()

>CP045149.1 Yersinia pestis strain SCPM-O-DNA-18 (I-3113) chromosome, complete genome


In [3]:
search_term = "Yersinia pestis strain SCPM-O-DNA-18 (I-3113) chromosome, complete genome"
handle = Entrez.esearch(db='nucleotide', term=search_term, retmax=20)
result = Entrez.read(handle)
handle.close()
print(result)

{'Count': '3', 'RetMax': '3', 'RetStart': '0', 'IdList': ['1769169000', '1769093841', '1769089848'], 'TranslationSet': [{'From': 'Yersinia pestis', 'To': '"Yersinia pestis"[Organism] OR Yersinia pestis[All Fields]'}], 'TranslationStack': [{'Term': '"Yersinia pestis"[Organism]', 'Field': 'Organism', 'Count': '456940', 'Explode': 'Y'}, {'Term': 'Yersinia pestis[All Fields]', 'Field': 'All Fields', 'Count': '472721', 'Explode': 'N'}, 'OR', 'GROUP', {'Term': 'strain[All Fields]', 'Field': 'All Fields', 'Count': '133686779', 'Explode': 'N'}, 'AND', {'Term': 'SCPM-O-DNA-18[All Fields]', 'Field': 'All Fields', 'Count': '10', 'Explode': 'N'}, 'AND', 'GROUP', {'Term': 'I-3113[All Fields]', 'Field': 'All Fields', 'Count': '13', 'Explode': 'N'}, 'AND', {'Term': 'chromosome[All Fields]', 'Field': 'All Fields', 'Count': '71214746', 'Explode': 'N'}, {'Term': 'complete[All Fields]', 'Field': 'All Fields', 'Count': '48680910', 'Explode': 'N'}, 'AND', {'Term': 'genome[All Fields]', 'Field': 'All Fields

In [4]:
# let's take the first search result w/ Id '1769169000'

genome_id = '1769169000'
record = Entrez.efetch(db="nucleotide", id=genome_id, rettype="fasta", retmode="text")
filename = 'Sequence_{}.fasta'.format(genome_id)
print('Writing:{}'.format(filename))
with open(filename, 'w') as f:
  f.write(record.read())

Writing:Sequence_1769169000.fasta


Then we need to install EMBOSS.

In [1]:
!apt-get install emboss -q

Reading package lists...
Building dependency tree...
Reading state information...
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-11-0 cuda-nsig

In [22]:
#import wget
#emboss_url = 'ftp://emboss.open-bio.org/pub/EMBOSS/EMBOSS-6.6.0.tar.gz'
#emboss_filename = wget.download(emboss_url)

In [2]:
# untar/unzip EMBOSS

#!gunzip EMBOSS-6.6.0.tar.gz
#!tar xvf EMBOSS-6.6.0.tar

In [3]:
# EMBOSS compiling # ..

#!EMBOSS-6.6.0/configure --prefix=/usr/local/emboss
#!make EMBOSS-6.6.0/emboss/ 
#!export PATH=home/auser/EMBOSS-6.6.0/emboss:$PATH

In [32]:
!embossversion # to test all is well with your installation

/bin/bash: embossversion: command not found


In [38]:
# see the full length of this genome at first

!awk '/^>/{if (l!="") print l; print; l=0; next}{l+=length($0)}END{print l}' Sequence_1769169000.fasta

>NZ_CP045149.1 Yersinia pestis strain SCPM-O-DNA-18 (I-3113) chromosome, complete genome
4546217


The full length of the genome is 4546217.

In [39]:
import math

genome_length = 4546217
f = genome_length / 10 # it is needed for the next step [splitter]
fragment = math.ceil(f)
print(fragment)

454622


Let's cut the genome into ten parts by command _splitter_.

In [40]:
genome = "Sequence_1769169000.fasta"
!splitter -sequence {genome} -outseq "Sequence_1769169000_splitted.fasta" -size 454622 -ossingle2=Y
# we get ten files nz_cp045149.1_{nucletide_positions[start,end]} by use of -ossingle2 flag

Split sequence(s) into smaller sequences


Count a number of the following words: AAAA, ATAT, ATTA, AATT (by command _compseq_).

In [41]:
import os
import glob
from pathlib import Path

#!pwd -> /content
dir = '/content' # set directory w/ needed files
files = list(glob.glob(os.path.join(dir,'nz_cp045149.1*')))
fragment_files = []

for file in files:
  fragment_files.append(Path(file).stem)

fragment_files

['nz_cp045149.1_1-454622',
 'nz_cp045149.1_454623-909244',
 'nz_cp045149.1_1363867-1818488',
 'nz_cp045149.1_3636977-4091598',
 'nz_cp045149.1_2727733-3182354',
 'nz_cp045149.1_2273111-2727732',
 'nz_cp045149.1_3182355-3636976',
 'nz_cp045149.1_1818489-2273110',
 'nz_cp045149.1_4091599-4546217',
 'nz_cp045149.1_909245-1363866']

In [42]:
k = 4
for fragment_file in fragment_files:
    !compseq -sequence {fragment_file}.fasta -word {k} -outfile {fragment_file}_{k}-mer.fasta

Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences


In [43]:
words = ["AAAA", "ATAT", "ATTA", "AATT"]

files = list(glob.glob(os.path.join(dir,'*-mer.fasta')))
fragmented_files = []

for file in files:
  fragmented_files.append(os.path.basename(file))

fragmented_files

['nz_cp045149.1_3636977-4091598_4-mer.fasta',
 'nz_cp045149.1_3182355-3636976_4-mer.fasta',
 'nz_cp045149.1_454623-909244_4-mer.fasta',
 'nz_cp045149.1_2727733-3182354_4-mer.fasta',
 'nz_cp045149.1_1363867-1818488_4-mer.fasta',
 'nz_cp045149.1_2273111-2727732_4-mer.fasta',
 'nz_cp045149.1_4091599-4546217_4-mer.fasta',
 'nz_cp045149.1_1818489-2273110_4-mer.fasta',
 'nz_cp045149.1_1-454622_4-mer.fasta',
 'nz_cp045149.1_909245-1363866_4-mer.fasta']

In [44]:
times1 = [] # we need this but later
for fragmented_file in fragmented_files:
  for word in words:
    for line in open(fragmented_file):
      if word in line:
        times = line[5:9] # ~
        times1.append(times)
        print(fragment_file, "contains", word, times, "times")

nz_cp045149.1_909245-1363866 contains AAAA 3763 times
nz_cp045149.1_909245-1363866 contains ATAT 2291 times
nz_cp045149.1_909245-1363866 contains ATTA 2268 times
nz_cp045149.1_909245-1363866 contains AATT 2337 times
nz_cp045149.1_909245-1363866 contains AAAA 3764 times
nz_cp045149.1_909245-1363866 contains ATAT 2624 times
nz_cp045149.1_909245-1363866 contains ATTA 2616 times
nz_cp045149.1_909245-1363866 contains AATT 2392 times
nz_cp045149.1_909245-1363866 contains AAAA 3287 times
nz_cp045149.1_909245-1363866 contains ATAT 2217 times
nz_cp045149.1_909245-1363866 contains ATTA 2282 times
nz_cp045149.1_909245-1363866 contains AATT 2346 times
nz_cp045149.1_909245-1363866 contains AAAA 3871 times
nz_cp045149.1_909245-1363866 contains ATAT 2340 times
nz_cp045149.1_909245-1363866 contains ATTA 2490 times
nz_cp045149.1_909245-1363866 contains AATT 2543 times
nz_cp045149.1_909245-1363866 contains AAAA 3899 times
nz_cp045149.1_909245-1363866 contains ATAT 2511 times
nz_cp045149.1_909245-1363866

Then we need to mix each of these ten fragments by command _shuffleseq_.

In [45]:
for fragment_file in fragment_files:
  !shuffleseq -sequence {fragment_file}.fasta -outseq {fragment_file}_mixed.fasta

Shuffle a set of sequences maintaining composition
Shuffle a set of sequences maintaining composition
Shuffle a set of sequences maintaining composition
Shuffle a set of sequences maintaining composition
Shuffle a set of sequences maintaining composition
Shuffle a set of sequences maintaining composition
Shuffle a set of sequences maintaining composition
Shuffle a set of sequences maintaining composition
Shuffle a set of sequences maintaining composition
Shuffle a set of sequences maintaining composition


In [46]:
files = list(glob.glob(os.path.join(dir,'*_mixed.fasta')))
mixed_files = []

for file in files:
  mixed_files.append(Path(file).stem)

mixed_files

['nz_cp045149.1_3636977-4091598_mixed',
 'nz_cp045149.1_4091599-4546217_mixed',
 'nz_cp045149.1_3182355-3636976_mixed',
 'nz_cp045149.1_2273111-2727732_mixed',
 'nz_cp045149.1_454623-909244_mixed',
 'nz_cp045149.1_2727733-3182354_mixed',
 'nz_cp045149.1_1363867-1818488_mixed',
 'nz_cp045149.1_1-454622_mixed',
 'nz_cp045149.1_1818489-2273110_mixed',
 'nz_cp045149.1_909245-1363866_mixed']

Let's count again a number of the following words: AAAA, ATAT, ATTA, AATT (but for now in the mixed fragments).

In [47]:
for mixed_file in mixed_files:
    !compseq -sequence {mixed_file}.fasta -word {k} -outfile {mixed_file}_{k}-mer.fasta

Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences
Calculate the composition of unique words in sequences


In [48]:
files = list(glob.glob(os.path.join(dir,'*_mixed_4-mer.fasta')))
mixedd_files = []

for file in files:
  mixedd_files.append(os.path.basename(file))

mixedd_files

['nz_cp045149.1_1-454622_mixed_4-mer.fasta',
 'nz_cp045149.1_909245-1363866_mixed_4-mer.fasta',
 'nz_cp045149.1_454623-909244_mixed_4-mer.fasta',
 'nz_cp045149.1_1818489-2273110_mixed_4-mer.fasta',
 'nz_cp045149.1_1363867-1818488_mixed_4-mer.fasta',
 'nz_cp045149.1_4091599-4546217_mixed_4-mer.fasta',
 'nz_cp045149.1_2727733-3182354_mixed_4-mer.fasta',
 'nz_cp045149.1_3182355-3636976_mixed_4-mer.fasta',
 'nz_cp045149.1_2273111-2727732_mixed_4-mer.fasta',
 'nz_cp045149.1_3636977-4091598_mixed_4-mer.fasta']

In [50]:
times2 = [] # we need this but later, too
for mixedd_file in mixedd_files:
  for word in words:
    for line in open(mixedd_file):
      if word in line:
        times = line[5:9] # ~
        times2.append(times)
        print(mixedd_file, "contains", word, times, "times")

nz_cp045149.1_1-454622_mixed_4-mer.fasta contains AAAA 1817 times
nz_cp045149.1_1-454622_mixed_4-mer.fasta contains ATAT 2062 times
nz_cp045149.1_1-454622_mixed_4-mer.fasta contains ATTA 2052 times
nz_cp045149.1_1-454622_mixed_4-mer.fasta contains AATT 2029 times
nz_cp045149.1_909245-1363866_mixed_4-mer.fasta contains AAAA 1892 times
nz_cp045149.1_909245-1363866_mixed_4-mer.fasta contains ATAT 2180 times
nz_cp045149.1_909245-1363866_mixed_4-mer.fasta contains ATTA 2194 times
nz_cp045149.1_909245-1363866_mixed_4-mer.fasta contains AATT 2218 times
nz_cp045149.1_454623-909244_mixed_4-mer.fasta contains AAAA 1905 times
nz_cp045149.1_454623-909244_mixed_4-mer.fasta contains ATAT 1998 times
nz_cp045149.1_454623-909244_mixed_4-mer.fasta contains ATTA 2107 times
nz_cp045149.1_454623-909244_mixed_4-mer.fasta contains AATT 2056 times
nz_cp045149.1_1818489-2273110_mixed_4-mer.fasta contains AAAA 2293 times
nz_cp045149.1_1818489-2273110_mixed_4-mer.fasta contains ATAT 2239 times
nz_cp045149.1_1818

Then, we need to compare these two results related to the kmers counts (comparison should be implemented among them + expectected one according to the Bernoulli model).

The Bernoulli's theorem can be formulated as it follows: "If an unlimited number of tests are performed, and for all of these tests some events have the same probability, then, in the case of a sufficiently large number of these events, it can be claimed with a probability somehow/randomly close to the certainty that the ratio of the number of occurrences of an event to the number of tests will deviate from the probability of an event by less than a given number, no matter how small it can be".

Bernoulli's name is also associated with the classical definition of probability, according to which the probability of a random event is the ratio of the number of the favorable cases to the occurrence of an event (m) to the number of all "equally possible" cases (n): p = m/n.

According to the number of nucleotides that may exist in a genome (A, T, G, C) (we don't speak about RNA right now), we may calculate how many variances of k-mers (k=4) exist.

In [51]:
# 4 - A, T, G, C
n = 4**4
n

256

In [52]:
p = 1/n # probability to have a word (for instance, AAAA) in a genome
p

0.00390625

It is named as the Bernoulli model: P(ATGG) = P(A)xP(T)xP(G)xP(G).

So, the full length of the given genome is 4546217 (as we have recognized before); therefore, the expected number for each 4-letters word to exist in this genome is:

In [53]:
m_expected_average = (genome_length/10)*p # cause we cut it into ten parts
m_expected_average

1775.866015625

In [54]:
times_1 = list(map(int, times1))
times_1[:10]

[3763, 2291, 2268, 2337, 3764, 2624, 2616, 2392, 3287, 2217]

In [55]:
mean_1 = sum(times_1)/len(times_1)
mean_1 # 2750.425

2750.425

In [56]:
times_2 = list(map(int, times2))
times_2[:10]

[1817, 2062, 2052, 2029, 1892, 2180, 2194, 2218, 1905, 1998]

In [57]:
mean_2 = sum(times_2)/len(times_2)
mean_2 # 2664.75

2136.65

We may see that the mean 1 and the mean 2 of these two practical results are not similar (2750.425, 2136.65), while the expected number (1775.866) is much less both of them. 

###Uniprot

We have the protein "Aconitase" that should be analysed then.

The Uniprot website: https://www.uniprot.org

1. How many results are found in UniprotKB/Swiss-Prot/TrEMBL (using the protein name while searching)? / These three ones are considered as the main databases of UniProt.

Manually, by UniprotKB we get 160,879 results, by Swiss-Prot - 1,262, by TrEMBL - 159,617.


2. How many results are found in UniprotKB/Swiss-Prot/TrEMBL .. using advanced search method (the field - "protein name")?

Manually, by UniprotKB we get 31,272 results, by Swiss-Prot - 84, by TrEMBL - 31,188.



3. How many results are found in UniprotKB/Swiss-Prot/TrEMBL .. using advanced search method (the field - "protein name") + if we include the filter in the field "Taxonomy" (Homo sapiens).

Manually, by UniprotKB we get 12 results, by Swiss-Prot - 2, by TrEMBL - 10.


Now let's consider the info about the protein..

I have opened the tab w/ the entry name "ACON_HUMAN" (https://www.uniprot.org/uniprot/Q99798)

4. What's the protein function?

"Catalyzes the isomerization of citrate to isocitrate via cis-aconitate."

5. Which family does it belong to?

"Belongs to the aconitase/IPM isomerase family."

6. How many UniRef clusters with identities 1.0, 0.9, 0.5 does the protein belong to (including the isoforms)?

For 1.0 - 1, 
0.9 - 563, 
0.5 - 592.
